In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
"""This notebook creates the directory tree of TAG in COUNTRY 
in YEAR"""

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot import pagegenerators, textlib
    from pywikibot.specialbots import UploadRobot

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot import pagegenerators, textlib
    from pywikibot.specialbots import UploadRobot

import mwparserfromhell as mwh

In [ ]:
from anytree import Node, LevelOrderIter, RenderTree, AsciiStyle
from mako.template import Template

In [ ]:
# Project parameters
YEAR                = 2018
TAG                 = 'WLE'
TAG_EXT             = "Wiki Loves Earth"
COUNTRY             = "Spain"
ES_COUNTRY          = "España"

BASE_NAME           = "Commons:Wiki Loves in {2}/{1}/{0}".format(YEAR, TAG_EXT, COUNTRY)
commons_site = pb.Site('commons', 'commons')

In [ ]:
class TabularDataPage(pb.Page):
    def __init__(self, source, title=''):
        if not isinstance(source, pb.site.BaseSite):
            site = source.site
        else:
            site = source
        super(TabularDataPage, self).__init__(source, title)
        if self.namespace() != 486:
            raise ValueError('Page %s must belong to %d namespace'
                             % (self.title(), 486))

    def text(self, value):
        self._text = value

    def _page_to_json(self):
        return json.dumps(_text, ensure_ascii=False)
    
    def save(self, *args, **kwargs):  # See Page.save().
        """Save page content after recomposing the page."""
        # Save using contentformat='application/json'.
        kwargs['contentformat'] = 'application/json'
        _dict = {
            "license": "CC0-1.0",
            "schema": {
                "fields": [
                    {
                        "name": "date",
                        "type": "string"
                    }
                ]
            },
            "data": [
                [
                    "1876-01"
                ]
            ]
        }
        #kwargs['contentmodel'] = 'Tabular.JsonConfig'
        super(TabularDataPage, self).save(*args, text=json.dumps(_dict, ensure_ascii=False), **kwargs)

tabdata_page = TabularDataPage(commons_site, 'Data:Wiki Loves in Spain/Wiki Loves Earth/2017/Log_db.tab')
tabdata_page.text = ''
#tabdata_page.save("{0} in Spain report".format(TAG))

In [ ]:
root_template = """{{see also|Commons:${tag_extended} ${year}}}
{{es|1=Esta es la categoría principal del concurso fotográfico '''[[Commons:${tag_extended} ${year} in ${country}|${tag_extended} ${year} en ${es_country}]]'''.}}
[[Category:${tag_extended} ${year}|${country}]]
[[Category:${tag_extended} in ${country}| ${year}]]
[[Category:${year} events in ${country}]]
[[Category:Activities by Wikimedia ${es_country} in ${year}]]"""

vars = {
    "tag_extended": TAG_EXT,
    "year": YEAR,
    "country": COUNTRY,
    "es_country": ES_COUNTRY,
    "WM_CHAPTER": None
}
t = Template(root_template)
root_text = t.render(**vars)
root_text

In [ ]:
page = pb.Page(commons_site, "{0} {1} in {2}".format(TAG_EXT, YEAR, COUNTRY), ns=14)
if page.text != root_text:
    page.text = root_text
    page.save("{1} {0} in {2}: Folder structure creation".format(YEAR, TAG, COUNTRY))

In [ ]:
images_root_template = """{{Countries of Europe|prefix=:Category:Images from ${tag_extended} ${year} in}}

{{see also|Commons${tag_extended} ${year} in ${country}}}
{{Image template notice|${tag_extended} ${year}|es}}
{{GeoGroupTemplate}}
{{CategoryTOC}}
{{Hiddencat}}
<center><big>'''El concurso fotogáfico [[Commons:${tag_extended} ${year} in ${country}|${tag_extended} ${year} (${country})]] se celebra entre el 1 y el 31 de mayo de ${year}. Por favor, no incluyáis aquí imágenes que se subieron antes o después del plazo del concurso.'''</big></center>

[[Category:${tag_extended} ${year} in ${country}]]
[[Category:Photographs of ${country}|${tag_extended} ${year}]]
[[Category:Images from ${tag_extended} ${year}|${country}]]
[[Category:Images from ${tag_extended} in ${country} by year| ${year}]]"""

vars = {
    "tag_extended": TAG_EXT,
    "year": YEAR,
    "country": COUNTRY,
    "WM_CHAPTER": None
}
t = Template(images_root_template)
images_root_text = t.render(**vars)
images_root_text

In [ ]:
regular_template = """{{{{hiddencat}}}}

{0}
"""

In [ ]:
header_meta = "{{{{metacat|{0}}}}}\n"
header_categorize = "{{categorize}}\n"
header_featured = "{{Collection of featured pictures|country=es}}\n"
header_quality = """{{Countries of Europe|prefix=:Category:Quality images from Wiki Loves Earth 2015 in}}

{{Collection of quality images|country=es}}
{{CategoryTOC}}
"""
header_valued = """{{VI-cat}}
{{CategoryTOC}}
"""

In [ ]:
root           = Node("Images from {0} {1} in {2}".format(TAG_EXT, YEAR, COUNTRY), text=images_root_text)

maintenance    = Node("Images from {0} {1} in {2} (maintenance)".format(TAG_EXT, YEAR, COUNTRY), 
                      parent=root, 
                      order=0,
                      header=header_categorize)
evaluation     = Node("Images from {0} {1} in {2} (evaluation)".format(TAG_EXT, YEAR, COUNTRY), 
                      parent=root, 
                      order=2,
                      header=header_categorize)
classification = Node("Classified images from {0} {1} in {2}".format(TAG_EXT, YEAR, COUNTRY), 
                      parent=root, 
                      order=1,
                      header=header_categorize)
remarkable     = Node("High quality images from {0} {1} in {2}".format(TAG_EXT, YEAR, COUNTRY), 
                      parent=root, 
                      order=3,
                      header=header_categorize)

issues         = Node("Images with issues from {0} {1} in {2}".format(TAG_EXT, YEAR, COUNTRY),
                      parent=maintenance,
                      order=0,
                      header=header_categorize)
unqualified    = Node("Unqualified images from {0} {1} in {2}".format(TAG_EXT, YEAR, COUNTRY),
                      parent=maintenance,
                      order=1)

wrong_code  = Node("Images from {0} {1} in {2} with a wrong code".format(TAG_EXT, YEAR, COUNTRY), parent=issues)
no_code     = Node("Images from {0} {1} in {2} without code".format(TAG_EXT, YEAR, COUNTRY), parent=issues)
no_template = Node("Images from {0} {1} in {2} without valid template".format(TAG_EXT, YEAR, COUNTRY), parent=issues)
no_category = Node("Uncategorized images from {0} {1} in {2}".format(TAG_EXT, YEAR, COUNTRY), parent=issues)

no_site     = Node("Unqualified images from {0} {1} in {2} (not from a site of community importance)".format(TAG, YEAR, COUNTRY), 
                   parent=unqualified)
no_country  = Node("Unqualified images from {0} {1} in {2} (not from {2})".format(TAG, YEAR, COUNTRY), 
                   parent=unqualified)
no_date     = Node("Unqualified images from {0} {1} in {2} (wrong submission time)".format(TAG, YEAR, COUNTRY), 
                   parent=unqualified)
no_size     = Node("Unqualified images from {0} {1} in {2} (too small)".format(TAG, YEAR, COUNTRY), 
                   parent=unqualified)
no_location = Node("Unqualified images from {0} {1} in {2} (unidentified locations)".format(TAG, YEAR, COUNTRY), 
                   parent=unqualified)
watermarked = Node("Unqualified images from {0} {1} in {2} (watermarked)".format(TAG, YEAR, COUNTRY), 
                   parent=unqualified)

to_evaluate = Node("Images from {0} {1} in {2} to be evaluated".format(TAG_EXT, YEAR, COUNTRY), 
                   parent=evaluation,
                   order=0)
first_round = Node("Evaluation of images from {0} {1} in {2} - Round 1".format(TAG_EXT, YEAR, COUNTRY), 
                   parent=evaluation,
                   order=1)
finalist    = Node("Evaluation of images from {0} {1} in {2} - Final".format(TAG_EXT, YEAR, COUNTRY), 
                   parent=evaluation,
                   cats=["Evaluation of images from {0} in {2} - Final| {1}".format(TAG_EXT, YEAR, COUNTRY)],
                   order=3)
winners     = Node("Winners of {0} {1} in {2}".format(TAG_EXT, YEAR, COUNTRY), 
                   parent=evaluation,
                   cats=["Winners of {0} in {2}| {1}".format(TAG_EXT, YEAR, COUNTRY),
                         "Winners of {0} {1} by country|{2}".format(TAG_EXT, YEAR, COUNTRY)
                        ],
                   order=4)

by_author = Node("Images from {0} {1} in {2} by author".format(TAG_EXT, YEAR, COUNTRY), 
                 parent=classification, 
                 cats=["Images from {0} in {2} by author| {1}".format(TAG_EXT, YEAR, COUNTRY)],
                 order='a',
                 header=header_meta.format("author"))
by_autcom = Node("Images from {0} {1} in {2} by autonomous community".format(TAG_EXT, YEAR, COUNTRY), 
                 parent=classification,
                 cats=["Images from {0} in {2} by autonomous community| {1}".format(TAG_EXT, YEAR, COUNTRY)],
                 order='c',
                 header=header_meta.format("autonomous community"))
by_site   = Node("Images from {0} {1} in {2} by site".format(TAG_EXT, YEAR, COUNTRY), 
                 parent=classification,
                 cats=["Images from {0} in {2} by site and year| {1}".format(TAG_EXT, YEAR, COUNTRY)],
                 order='b',
                 header=header_meta.format("site"))

featured = Node("Featured images from {0} {1} in {2}".format(TAG_EXT, YEAR, COUNTRY), 
                parent=remarkable,
                cats=["Featured pictures from {0} in {2}| {1}".format(TAG_EXT, YEAR, COUNTRY),
                      "Featured pictures from {0} {1}|{2}".format(TAG_EXT, YEAR, COUNTRY),
                      "Featured pictures supported by Wikimedia España",
                     ],
                order=0,
                header=header_featured)
quality  = Node("Quality images from {0} {1} in {2}".format(TAG_EXT, YEAR, COUNTRY), 
                parent=remarkable,
                cats=["Quality images from {0} in {2}| {1}".format(TAG_EXT, YEAR, COUNTRY),
                      "Quality images from {0} {1}|{2}".format(TAG_EXT, YEAR, COUNTRY),
                      "Quality images supported by Wikimedia España"
                     ],
                order=1,
                header=header_quality)
valued   = Node("Valued images from {0} {1} in {2}".format(TAG_EXT, YEAR, COUNTRY), 
                parent=remarkable,
                cats=["Valued images from {0} in {2}| {1}".format(TAG_EXT, YEAR, COUNTRY),
                      "Valued images from {0} {1}|{2}".format(TAG_EXT, YEAR, COUNTRY),
                      "Valued images of {2}|{0} {1}".format(TAG_EXT, YEAR, COUNTRY)
                     ],
                order=2,
                header=header_valued)

In [ ]:
for pre, fill, node in RenderTree(root):
    print("%s%s" % (pre, node.name))

In [ ]:
for node in LevelOrderIter(root) :
    cat_wikitext = None
    try:
        try :
            # Parent category when there's a category key
            cats = ["{0}| {1}".format(node.parent.name, node.order)]
        except :
            # Parent category when there isn't a category key
            cats = [node.parent.name]
            
        try:
            # Include additional cats
            cats.extend(node.cats)
        except :
            pass
        # Create category string and include it into template throuhg format
        cat_text = '\n'.join(["[[Category:{}]]".format(cat) for cat in cats])
        cat_wikitext = regular_template.format(cat_text)
    except :
        # Root category
        cat_wikitext = node.text
        
    try :
        cat_wikitext = node.header + cat_wikitext
    except :
        pass
    
    page = pb.Page(commons_site, node.name, ns=14)

    if page.text.strip() != cat_wikitext.strip():
        page.text = cat_wikitext
        page.save("{1} {0} in {2}: Folder structure creation".format(YEAR, TAG, COUNTRY))
    #print (cat_wikitext)
    #print ("---------------------")